# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities2.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,12.72,67,45,3.37,SH,1734225914
1,1,port mathurin,-19.6833,63.4167,25.38,74,41,7.97,MU,1734226125
2,2,hadibu,12.6500,54.0333,23.82,79,59,4.93,YE,1734226126
3,3,kingston,17.9970,-76.7936,22.34,85,0,2.00,JM,1734225499
4,4,san pedro pochutla,15.7333,-96.4667,31.84,84,100,1.51,MX,1734226128


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_all = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_all

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 29
ideal_humidity = city_data_df["Humidity"] < 80

# Drop any rows with null values
ideal_city_df = city_data_df[min_temp_s & max_temp_s & ideal_humidity]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display a sample of the filtered data
ideal_city_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,port mathurin,-19.6833,63.4167,25.38,74,41,7.97,MU,1734226125
2,2,hadibu,12.6500,54.0333,23.82,79,59,4.93,YE,1734226126
7,7,wailua homesteads,22.0669,-159.3780,25.63,79,40,6.17,US,1734226132
12,12,blackmans bay,-43.0167,147.3167,21.40,61,6,0.89,AU,1734226137
32,32,hithadhoo,-0.6000,73.0833,27.79,79,9,7.20,MV,1734226160


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,port mathurin,MU,-19.6833,63.4167,74,
2,hadibu,YE,12.6500,54.0333,79,
7,wailua homesteads,US,22.0669,-159.3780,79,
12,blackmans bay,AU,-43.0167,147.3167,61,
32,hithadhoo,MV,-0.6000,73.0833,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

params = {
   "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5480d716fce048c9885704a4c21128e3&filter=circle%3A63.4167%2C-19.6833%2C10000&bias=proximity%3A63.4167%2C-19.6833
port mathurin - nearest hotel: Escale Vacances
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5480d716fce048c9885704a4c21128e3&filter=circle%3A54.0333%2C12.65%2C10000&bias=proximity%3A54.0333%2C12.65
hadibu - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5480d716fce048c9885704a4c21128e3&filter=circle%3A-159.378%2C22.0669%2C10000&bias=proximity%3A-159.378%2C22.0669
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5480d716fce048c9885704a4c21128e3&filter=circle%3A147.3167%2C-43.0167%2C10000&bias=proximity%3A147.3167%2C-43.0167
blackmans bay - nearest hotel: Villa Howden
https://ap

,City,Country,Lat,Lng,Humidity,Hotel Name
1,port mathurin,MU,-19.6833,63.4167,74,Escale Vacances
2,hadibu,YE,12.6500,54.0333,79,No hotel found
7,wailua homesteads,US,22.0669,-159.3780,79,Hilton Garden Inn Kauai Wailua Bay
12,blackmans bay,AU,-43.0167,147.3167,61,Villa Howden
32,hithadhoo,MV,-0.6000,73.0833,79,Pebbles Inn
...,...,...,...,...,...,...
539,alofi,NU,-19.0595,-169.9187,74,Matavai Resort
543,valenca do piaui,BR,-6.4075,-41.7456,47,No hotel found
550,beira,MZ,-19.8436,34.8389,74,Hotel Savoy
557,monteros,AR,-27.1674,-65.4983,66,Hotel Congreso


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City",
    hover_cols=["Hotel Name", "Country"], 
)

# Display the map plot
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)